# Importing libraries

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import skimage
from skimage import transform
from tensorflow import keras
from skimage import color

# Function for calcualting the accuracy of the classification

In [0]:
def acc(output,label):
    test=((output[:,0]-label)==0)*1.0
    return sum(test)/len(test)*100

# Sigmoid activation function

In [0]:
def sigmoid(x):
    return(1/(1+np.exp(-0.0001*x)))

## ReLu function

In [0]:
def relu(x):
  if x >0:
    return x
  else:
    return 0.1*x

In [0]:
def d_relu(x):
  if x >0:
    return 1
  else:
    return 0.1

# Network inputs

Loading data (Digits)

In [0]:
ix=28*28 # Number of features
epochs=20
digits=[0,1] # Digits to classify

# Loading data
mnist=keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


# Training

indices=np.append(np.where(train_labels==digits[0]),np.where(train_labels==digits[1]))
train=train_images[indices]
train=train.reshape(len(train),28*28)
# Additional column is added to the training set instead of calculating biases
train=np.concatenate((train,np.ones([len(train),1])),axis=1)
train_lbl=train_labels[indices]
train_lbl=(train_lbl==digits[1])*1.0

# Test

indices=np.append(np.where(test_labels==digits[0]),np.where(test_labels==digits[1]))
test=test_images[indices]
test=test.reshape(len(test),28*28)
test=np.concatenate((test,np.ones([len(test),1])),axis=1)
test_lbl=test_labels[indices]
test_lbl=(test_lbl==digits[1])*1.0

# Normalization
train = (train - np.min(train)) / (np.max(train) - np.min(train))
test = (test - np.min(test)) / (np.max(test) - np.min(test))


Loading data Cifar10

In [0]:
ix=32*32 # Number of features
epochs=40
digits=[0,1] # Digits to classify

# Loading data
mnist=keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_labels=train_labels.reshape(len(train_labels))
test_labels=test_labels.reshape(len(test_labels))

# Training

indices=np.append(np.where(train_labels==digits[0]),np.where(train_labels==digits[1]))
train=train_images[indices]
train=skimage.color.rgb2gray(train)
train=train.reshape(len(train),ix)
train=np.concatenate((train,np.ones([len(train),1])),axis=1)
train_lbl=train_labels[indices]
train_lbl=(train_lbl==digits[1])*1.0

# Test

indices=np.append(np.where(test_labels==digits[0]),np.where(test_labels==digits[1]))
test=test_images[indices]
test=skimage.color.rgb2gray(test)
test=test.reshape(len(test),ix)
test=np.concatenate((test,np.ones([len(test),1])),axis=1)
test_lbl=test_labels[indices]
test_lbl=(test_lbl==digits[1])*1.0

# Normalization
train = (train - np.min(train)) / (np.max(train) - np.min(train))
test = (test - np.min(test)) / (np.max(test) - np.min(test))


# Sigmoid Mnist

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]
print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=sigmoid(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*logits*(1-logits) # new
        w=w+diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 4376.14 accuracy: 99.61
Epoch: 1, error: 2707.54 accuracy: 99.64
Epoch: 2, error: 2088.15 accuracy: 99.65
Epoch: 3, error: 1752.88 accuracy: 99.66
Epoch: 4, error: 1535.80 accuracy: 99.65
Epoch: 5, error: 1382.76 accuracy: 99.67
Epoch: 6, error: 1266.76 accuracy: 99.68
Epoch: 7, error: 1174.94 accuracy: 99.69
Epoch: 8, error: 1100.83 accuracy: 99.69
Epoch: 9, error: 1038.56 accuracy: 99.69
Epoch: 10, error: 985.77 accuracy: 99.70
Epoch: 11, error: 940.36 accuracy: 99.71
Epoch: 12, error: 900.35 accuracy: 99.72
Epoch: 13, error: 865.08 accuracy: 99.72
Epoch: 14, error: 833.82 accuracy: 99.72
Epoch: 15, error: 805.28 accuracy: 99.73
Epoch: 16, error: 779.91 accuracy: 99.73
Epoch: 17, error: 756.49 accuracy: 99.73
Epoch: 18, error: 735.09 accuracy: 99.73
Epoch: 19, error: 715.39 accuracy: 99.73

Test:

Test accuracy: 99.91


## ReLu Mnist

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]

print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=relu(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*d_relu(logits) # new
        w=w+0.01*diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 5512.24 accuracy: 97.07
Epoch: 1, error: 2136.31 accuracy: 97.16
Epoch: 2, error: 1668.37 accuracy: 97.39
Epoch: 3, error: 1468.86 accuracy: 98.81
Epoch: 4, error: 1315.34 accuracy: 97.68
Epoch: 5, error: 1229.22 accuracy: 97.91
Epoch: 6, error: 1177.14 accuracy: 95.65
Epoch: 7, error: 1110.23 accuracy: 98.88
Epoch: 8, error: 1075.77 accuracy: 95.90
Epoch: 9, error: 1038.76 accuracy: 98.16
Epoch: 10, error: 1020.83 accuracy: 96.63
Epoch: 11, error: 989.34 accuracy: 96.64
Epoch: 12, error: 967.97 accuracy: 98.57
Epoch: 13, error: 954.76 accuracy: 98.52
Epoch: 14, error: 942.80 accuracy: 98.84
Epoch: 15, error: 922.39 accuracy: 95.67
Epoch: 16, error: 910.85 accuracy: 98.02
Epoch: 17, error: 901.62 accuracy: 99.24
Epoch: 18, error: 889.97 accuracy: 98.93
Epoch: 19, error: 886.88 accuracy: 97.99

Test:

Test accuracy: 98.20


## Sigmoid Cifar 10

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]
print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=sigmoid(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*logits*(1-logits) # new
        w=w+diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 4880.47 accuracy: 54.34
Epoch: 1, error: 4732.77 accuracy: 69.39
Epoch: 2, error: 4610.38 accuracy: 71.15
Epoch: 3, error: 4506.14 accuracy: 72.73
Epoch: 4, error: 4420.85 accuracy: 73.31
Epoch: 5, error: 4346.05 accuracy: 74.17
Epoch: 6, error: 4279.44 accuracy: 74.27
Epoch: 7, error: 4226.12 accuracy: 74.38
Epoch: 8, error: 4176.71 accuracy: 74.62
Epoch: 9, error: 4131.24 accuracy: 74.80
Epoch: 10, error: 4091.20 accuracy: 74.76
Epoch: 11, error: 4056.33 accuracy: 74.99
Epoch: 12, error: 4022.85 accuracy: 75.07
Epoch: 13, error: 3993.15 accuracy: 75.13
Epoch: 14, error: 3965.18 accuracy: 75.12
Epoch: 15, error: 3940.31 accuracy: 75.33
Epoch: 16, error: 3915.45 accuracy: 75.42
Epoch: 17, error: 3894.84 accuracy: 75.50
Epoch: 18, error: 3874.42 accuracy: 75.62
Epoch: 19, error: 3856.17 accuracy: 75.66
Epoch: 20, error: 3837.22 accuracy: 75.70
Epoch: 21, error: 3818.43 accuracy: 75.73
Epoch: 22, error: 3804.59 accuracy: 75.80
Epoch: 23, error: 3788.57 accura

## ReLu Cifar 10

In [0]:
# Training
w=np.random.rand(ix+1,1) # Weights initialization
acmse=[]

print('\nTraining:\n')
for epoch in range(epochs):
    loss=0
    ind=random.sample(range(len(train)), len(train)) # new
    train=train[ind] # new
    train_lbl=train_lbl[ind] # new
    for i in range(len(train)):
        logits=np.einsum('i,ij->j',train[i],w)
        logits=relu(logits) # news
        error=train_lbl[i]-logits
        diffx=train[i]*d_relu(logits) # new
        w=w+0.01*diffx.reshape(diffx.shape[0],1)*error # substitute
        loss+=abs(error)

    # Training set accuracy
    logits=np.einsum('ij,jk->ik',train,w)
    logits=sigmoid(logits) # new
    logits=(logits>0.5)*1.0 # news
    acmse.append(acc(logits,train_lbl))
    print('Epoch: {:}, error: {:0.2f} accuracy: {:0.2f}'.format(epoch,loss[0],acc(logits,train_lbl)))


# Test set accuracy
print('\nTest:\n')
logits=np.einsum('ij,jk->ik',test,w)
logits=sigmoid(logits) # new
logits=(logits>0.5)*1.0 # new
print('Test accuracy: {:0.2f}'.format(acc(logits,test_lbl)))


Training:

Epoch: 0, error: 13919.75 accuracy: 50.02
Epoch: 1, error: 7637.72 accuracy: 57.13
Epoch: 2, error: 7286.33 accuracy: 67.03
Epoch: 3, error: 6868.70 accuracy: 56.59
Epoch: 4, error: 6770.22 accuracy: 51.00
Epoch: 5, error: 6594.94 accuracy: 56.35
Epoch: 6, error: 6689.64 accuracy: 49.99
Epoch: 7, error: 6572.53 accuracy: 72.27
Epoch: 8, error: 6523.33 accuracy: 61.15
Epoch: 9, error: 6633.52 accuracy: 50.01
Epoch: 10, error: 6497.38 accuracy: 70.47
Epoch: 11, error: 6521.09 accuracy: 50.50
Epoch: 12, error: 6391.90 accuracy: 50.79
Epoch: 13, error: 6527.73 accuracy: 58.02
Epoch: 14, error: 6480.14 accuracy: 53.09
Epoch: 15, error: 6630.80 accuracy: 50.23
Epoch: 16, error: 6373.28 accuracy: 64.77
Epoch: 17, error: 6538.50 accuracy: 50.09
Epoch: 18, error: 6517.64 accuracy: 59.90
Epoch: 19, error: 6658.68 accuracy: 50.72
Epoch: 20, error: 6497.23 accuracy: 62.48
Epoch: 21, error: 6577.88 accuracy: 72.53
Epoch: 22, error: 6540.35 accuracy: 58.79
Epoch: 23, error: 6432.97 accur